In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import urllib.request
import mecab
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import warnings
warnings.filterwarnings(action='ignore')

# 데이터 로드

In [2]:
df=pd.read_csv('./labelO_1922.csv')

In [3]:
# df[df['polarity.label']==-1]

# 연간

In [4]:
df["date"] = pd.to_datetime(df["date"])

In [5]:
import datetime as dt

In [6]:
year =[]
for dt in  df['date']:
    y=dt.year
    year.append(y)

In [7]:
month=[]
for dt in  df['date']:
    m=dt.month
    month.append(m)

In [8]:
df['year'] = year
df['month'] = month

In [9]:
corp =[]
for txt in df['corp']:
    name=txt.split(sep='-')[1]
    corp.append(name)

In [10]:
df['name'] = corp

In [11]:
df['pos']=0
df['neg']=0

In [12]:
for i,data in enumerate(df['polarity.label']):
    if data==1:
        df['pos'][i]=1
    elif data==-1:
        df['neg'][i]=-1
        # print(1)

In [13]:
df_2019 = df[df['year']==2019]
df_2020 = df[df['year']==2020]
df_2021 = df[df['year']==2021]
df_2022 = df[df['year']==2022]


In [14]:
df_2019[:2]

,date,category,section,publisher,author,title,content_url,attachment,industry.label,industry.score,...,polarity.score,polarity.name,corp,tokenized,del_stopword,year,month,name,pos,neg
0,2019-12-30,news,economy,한국경제TV,라이온봇,"YG PLUS, 주가 반등 현재는 +9.3%... 이 시각 50만3383주 거래",NaN,NaN,M71,1.000,...,0.900,긍정,DeepSearch-CMG제약-news-2019-01-01-2019-12-31-20...,NaN,NaN,2019,12,CMG제약,1,0
1,2019-12-18,news,economy,한국경제TV,라이온봇,"써니전자, 상승출발 후 현재 +6.2%... 외국인 5,883주 순매수",NaN,NaN,C26,0.999,...,0.569,긍정,DeepSearch-CMG제약-news-2019-01-01-2019-12-31-20...,NaN,NaN,2019,12,CMG제약,1,0


In [15]:
df_2019.iloc[:,[19,20,21]]

,name,pos,neg
0,CMG제약,1,0
1,CMG제약,1,0
2,CMG제약,1,0
3,CMG제약,1,0
4,CMG제약,0,0
...,...,...,...
73141,휴온스글로벌,0,0
73142,휴온스글로벌,0,0
73143,휴온스글로벌,0,0
73144,휴온스글로벌,0,0


In [16]:
df_2019 = df_2019.iloc[:,[19,20,21]]
df_2019 = df_2019.groupby('name').sum()
df_2019 = df_2019.reset_index()
df_2019['date'] = '2019'
df_2019 = df_2019[['name','date','pos','neg']]

df_2019.head(2)

,name,date,pos,neg
0,CMG제약,2019,40,-6
1,HLB,2019,10,0


In [17]:
df_2020 = df_2020.iloc[:,[19,20,21]]
df_2020 = df_2020.groupby('name').sum()
df_2020 = df_2020.reset_index()
df_2020['date'] = '2020'
df_2020 = df_2020[['name','date','pos','neg']]

df_2020.head(2)

,name,date,pos,neg
0,CMG제약,2020,15,-2
1,HLB,2020,11,0


In [18]:
df_2021 = df_2021.iloc[:,[19,20,21]]
df_2021 = df_2021.groupby('name').sum()
df_2021 = df_2021.reset_index()
df_2021['date'] = '2021'
df_2021 = df_2021[['name','date','pos','neg']]

df_2021.head(2)

,name,date,pos,neg
0,CMG제약,2021,14,-1
1,HLB,2021,12,-1


In [19]:
df_2022 = df_2022.iloc[:,[19,20,21]]
df_2022 = df_2022.groupby('name').sum()
df_2022 = df_2022.reset_index()
df_2022['date'] = '2022'
df_2022 = df_2022[['name','date','pos','neg']]

df_2022.head(2)

,name,date,pos,neg
0,CMG제약,2022,8,0
1,HLB생명과학,2022,40,-2


In [20]:
len(df_2019),len(df_2020),len(df_2021),len(df_2022)

(115, 118, 121, 118)

In [21]:
ticker = pd.read_csv('../데이터/최종기업ticker.csv',names=['name','ticker'])
ticker = ticker.dropna()

In [22]:
ticker[:2]

,name,ticker
1,CMG제약,058820
2,HLB,028300


In [23]:
merge_2019 = pd.merge(ticker,df_2019, how='left', left_on='name', right_on='name')
merge_2019 = merge_2019.drop('ticker',axis=1)
merge_2019['date'] = 2019
# merge_2019[10:20]

In [24]:
merge_2020 = pd.merge(ticker,df_2020, how='left', left_on='name', right_on='name')
merge_2020 = merge_2020.drop('ticker',axis=1)
merge_2020['date'] = 2020
# merge_2020[:-10]

In [25]:
merge_2021 = pd.merge(ticker,df_2021, how='left', left_on='name', right_on='name')
merge_2021 = merge_2021.drop('ticker',axis=1)
merge_2021['date'] = 2021
# merge_2021[40:50]

In [26]:
merge_2022 = pd.merge(ticker,df_2022, how='left', left_on='name', right_on='name')
merge_2022 = merge_2022.drop('ticker',axis=1)
merge_2022['date'] = 2022
# merge_2022[10:20]

In [27]:
df_2019['name']
news_연간_1922 = pd.concat([merge_2019,merge_2020,merge_2021,merge_2022])

In [28]:
news_연간_1922.to_csv('./news_연간(19-22).csv',index=False)

# 월간

In [29]:
df=pd.read_csv('./labelO_1922.csv')

In [30]:
# df[df['polarity.label']==-1]

In [31]:
df["date"] = pd.to_datetime(df["date"])

In [32]:
corp =[]
for txt in df['corp']:
    name=txt.split(sep='-')[1]
    corp.append(name)

In [33]:
df['name'] = corp

In [34]:
import datetime as dt

In [35]:
year =[]
for dt in  df['date']:
    y=dt.year
    year.append(y)

In [36]:
month=[]
for dt in  df['date']:
    m=dt.month
    month.append(m)

In [37]:
df['year'] = year
df['month'] = month

In [38]:
df['pos']=0
df['neg']=0

In [39]:
for i,data in enumerate(df['polarity.label']):
    if data==1:
        df['pos'][i]=1
    elif data==-1:
        df['neg'][i]=-1
        # print(1)

In [40]:
df_2019 = df[df['year']==2019]
df_2020 = df[df['year']==2020]
df_2021 = df[df['year']==2021]
df_2022 = df[df['year']==2022]


In [41]:
df_2019[:2]

,date,category,section,publisher,author,title,content_url,attachment,industry.label,industry.score,...,polarity.score,polarity.name,corp,tokenized,del_stopword,name,year,month,pos,neg
0,2019-12-30,news,economy,한국경제TV,라이온봇,"YG PLUS, 주가 반등 현재는 +9.3%... 이 시각 50만3383주 거래",NaN,NaN,M71,1.000,...,0.900,긍정,DeepSearch-CMG제약-news-2019-01-01-2019-12-31-20...,NaN,NaN,CMG제약,2019,12,1,0
1,2019-12-18,news,economy,한국경제TV,라이온봇,"써니전자, 상승출발 후 현재 +6.2%... 외국인 5,883주 순매수",NaN,NaN,C26,0.999,...,0.569,긍정,DeepSearch-CMG제약-news-2019-01-01-2019-12-31-20...,NaN,NaN,CMG제약,2019,12,1,0


In [42]:
dflist_19=[]
for i in range(1,13):
    df_i = df_2019[df_2019['month']==i]
    df_i = df_i.iloc[:,[17,19,20,21]]
    dflist_19.append(df_i)

In [43]:
dflist_20=[]
for i in range(1,13):
    df_i = df_2020[df_2020['month']==i]
    df_i = df_i.iloc[:,[17,19,20,21]]
    dflist_20.append(df_i)

In [44]:
dflist_21=[]
for i in range(1,13):
    df_i = df_2021[df_2021['month']==i]
    df_i = df_i.iloc[:,[17,19,20,21]]
    dflist_21.append(df_i)

In [45]:
dflist_22=[]
for i in range(1,13):
    df_i = df_2022[df_2022['month']==i]
    df_i = df_i.iloc[:,[17,19,20,21]]

    dflist_22.append(df_i)

In [46]:
# dflist_19[0]

In [47]:
dflist_19_1 = []
for df in dflist_19:
    tmp=df.groupby('name').sum()
    dflist_19_1.append(tmp)

In [48]:
dflist_20_1 = []
for df in dflist_20:
    tmp=df.groupby('name').sum()
    dflist_20_1.append(tmp)

In [49]:
dflist_21_1 = []
for df in dflist_21:
    tmp=df.groupby('name').sum()
    dflist_21_1.append(tmp)

In [50]:
dflist_22_1 = []
for df in dflist_22:
    tmp=df.groupby('name').sum()
    dflist_22_1.append(tmp)

In [51]:
dflist_22_1[0].head(2)

,month,pos,neg
name,,,
CMG제약,3,2,0
HLB생명과학,2,2,0


# 예쁘게

In [52]:
ticker[:2]

,name,ticker
1,CMG제약,058820
2,HLB,028300


In [53]:
# dflist_19_2[0]

In [54]:
dflist_19_2=[]
for i,df in enumerate(dflist_19_1):
    df_i = df.reset_index()
    df_i = df_i.drop('month', axis=1)
    df_i['date'] = f'2019-{i+1}'
    df_i = df_i[['name','date','pos','neg']]
    dflist_19_2.append(df_i)

In [55]:
dflist_20_2=[]
for i,df in enumerate(dflist_20_1):
    df_i = df.reset_index()
    df_i['date'] = f'2020-{i+1}'
    df_i = df_i[['name','date','pos','neg']]
    dflist_20_2.append(df_i)

In [56]:
dflist_21_2=[]
for i,df in enumerate(dflist_21_1):
    df_i = df.reset_index()
    df_i['date'] = f'2021-{i+1}'
    df_i = df_i[['name','date','pos','neg']]
    dflist_21_2.append(df_i)

In [57]:
dflist_22_2=[]
for i,df in enumerate(dflist_22_1):
    df_i = df.reset_index()
    df_i['date'] = f'2022-{i+1}'
    df_i = df_i[['name','date','pos','neg']]
    dflist_22_2.append(df_i)

In [58]:
dflist_22_2[0]

,name,date,pos,neg
0,CMG제약,2022-1,2,0
1,HLB생명과학,2022-1,2,0
2,JW생명과학,2022-1,0,0
3,JW신약,2022-1,6,0
4,JW중외제약,2022-1,2,-1
...,...,...,...,...
84,한미약품,2022-1,14,-2
85,한올바이오파마,2022-1,3,0
86,휴메딕스,2022-1,4,0
87,휴온스,2022-1,5,0


# merge - 없는 기업들도 df에 있기는 해야함

In [59]:
# merge_2019 = pd.merge(ticker,df_2019, how='left', left_on='name', right_on='name')
# merge_2019 = merge_2019.drop('ticker',axis=1)
# merge_2019['date'] = 2019
# # merge_2019[10:20]

In [60]:
dflist_19_3=[]
for i,df in enumerate(dflist_19_2):
    merge_df = pd.merge(ticker,df, how='left', left_on='name', right_on='name')
    merge_df = merge_df.drop('ticker',axis=1)
    merge_df['date'] = f'2019-{i+1}'
    dflist_19_3.append(merge_df)

In [61]:
dflist_19_3[5].head(2)

,name,date,pos,neg
0,CMG제약,2019-6,1.0,0.0
1,HLB,2019-6,2.0,0.0


In [62]:
dflist_20_3=[]
for i,df in enumerate(dflist_20_2):
    merge_df = pd.merge(ticker,df, how='left', left_on='name', right_on='name')
    merge_df = merge_df.drop('ticker',axis=1)
    merge_df['date'] = f'2020-{i+1}'
    dflist_20_3.append(merge_df)

In [63]:
dflist_21_3=[]
for i,df in enumerate(dflist_21_2):
    merge_df = pd.merge(ticker,df, how='left', left_on='name', right_on='name')
    merge_df = merge_df.drop('ticker',axis=1)
    merge_df['date'] = f'2021-{i+1}'
    dflist_21_3.append(merge_df)

In [64]:
dflist_22_3=[]
for i,df in enumerate(dflist_22_2):
    merge_df = pd.merge(ticker,df, how='left', left_on='name', right_on='name')
    merge_df = merge_df.drop('ticker',axis=1)
    merge_df['date'] = f'2022-{i+1}'
    dflist_22_3.append(merge_df)

In [65]:
len(dflist_22_3[5])

126

In [66]:
dflist_20_3[5].head(2)

,name,date,pos,neg
0,CMG제약,2020-6,NaN,NaN
1,HLB,2020-6,2.0,0.0


In [67]:
# dflist_19_3[0]

In [68]:
# dflist_19_4 = pd.DataFrame()
# for df in dflist_19_3:
#     dflist_19_4.append(df)

In [69]:
dflist_19_3 = pd.concat(dflist_19_3)
dflist_20_3 = pd.concat(dflist_20_3)
dflist_21_3 = pd.concat(dflist_21_3)
dflist_22_3 = pd.concat(dflist_22_3)


In [70]:
# dflist_19_3[:2]

In [71]:
news_월간_19_22 = pd.concat([dflist_19_3,dflist_20_3,dflist_21_3,dflist_22_3])

In [72]:
len(news_월간_19_22)

6048

In [73]:
news_월간_19_22.to_csv('./news_월간(19-22).csv', index=False)